<h2 style='color:purple' align='center'>Naive Bayes Assignment: Predicting survival from titanic crash</h2>

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("titanic.csv")
df.head()

PassengerId                                               Name  Pclass  \
0            1                            Braund, Mr. Owen Harris       3   
1            2  Cumings, Mrs. John Bradley (Florence Briggs Th...       1   
2            3                             Heikkinen, Miss. Laina       3   
3            4       Futrelle, Mrs. Jacques Heath (Lily May Peel)       1   
4            5                           Allen, Mr. William Henry       3   

   Gender   Age  SibSp  Parch            Ticket     Fare Cabin Embarked  \
0    male  22.0      1      0         A/5 21171   7.2500   NaN        S   
1  female  38.0      1      0          PC 17599  71.2833   C85        C   
2  female  26.0      0      0  STON/O2. 3101282   7.9250   NaN        S   
3  female  35.0      1      0            113803  53.1000  C123        S   
4    male  35.0      0      0            373450   8.0500   NaN        S   

   Survived  
0         0  
1         1  
2         1  
3         1  
4         0

In [3]:
df.drop(['PassengerId','Name','SibSp','Parch','Ticket','Cabin','Embarked'],axis='columns',inplace=True)
df.head()

Pclass  Gender   Age     Fare  Survived
0       3    male  22.0   7.2500         0
1       1  female  38.0  71.2833         1
2       3  female  26.0   7.9250         1
3       1  female  35.0  53.1000         1
4       3    male  35.0   8.0500         0

In [4]:
inputs = df.drop('Survived',axis='columns')
target = df.Survived

In [5]:
dummies = pd.get_dummies(inputs.Gender).astype(int)
dummies.head(3)

female  male
0       0     1
1       1     0
2       1     0

In [6]:
inputs = pd.concat([inputs,dummies],axis='columns')
inputs.head(3)

Pclass  Gender   Age     Fare  female  male
0       3    male  22.0   7.2500       0     1
1       1  female  38.0  71.2833       1     0
2       3  female  26.0   7.9250       1     0

**I am dropping male column as well because of dummy variable trap theory. One column is enough to repressent male vs female**
#### The Dummy Variable Trap occurs when two or more dummy variables created by one-hot encoding are highly correlated (multi-collinear).

In [7]:
inputs.drop(['Gender','male'],axis='columns',inplace=True)
inputs.head(3)

Pclass   Age     Fare  female
0       3  22.0   7.2500       0
1       1  38.0  71.2833       1
2       3  26.0   7.9250       1

In [8]:
inputs.columns[inputs.isna().any()]

Index(['Age'], dtype='object')

In [9]:
inputs.Age[:10]

0    22.0
1    38.0
2    26.0
3    35.0
4    35.0
5     NaN
6    54.0
7     2.0
8    27.0
9    14.0
Name: Age, dtype: float64

In [10]:
inputs.Age = inputs.Age.fillna(inputs.Age.mean())
inputs.head()

Pclass   Age     Fare  female
0       3  22.0   7.2500       0
1       1  38.0  71.2833       1
2       3  26.0   7.9250       1
3       1  35.0  53.1000       1
4       3  35.0   8.0500       0

In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(inputs,target,test_size=0.3)

In [12]:
# Gaussian Naive Bayes:
# Assumes that the features follow a normal distribution.
# It's suitable for continuous data.

# Multinomial Naive Bayes:
# Used for discrete data, like word count, movie rating (1 to 5)
# It's commonly used in text classification problems.

# Bernoulli Naive Bayes:
# Assumes that all the features are binary-valued (e.g., presence or absence of a feature).
# Also commonly used in text classification tasks, especially when dealing with binary features.

In [13]:
from sklearn.naive_bayes import GaussianNB
model = GaussianNB()

In [14]:
model.fit(X_train,y_train)

GaussianNB()

In [15]:
model.score(X_test,y_test)

0.7350746268656716

In [16]:
X_test[0:10]

Pclass        Age      Fare  female
577       1  39.000000   55.9000       1
889       1  26.000000   30.0000       0
297       1   2.000000  151.5500       1
786       3  18.000000    7.4958       1
353       3  25.000000   17.8000       0
598       3  29.699118    7.2250       0
111       3  14.500000   14.4542       1
715       3  19.000000    7.6500       0
796       1  49.000000   25.9292       1
672       2  70.000000   10.5000       0

In [17]:
y_test[0:10]

577    1
889    1
297    0
786    1
353    0
598    0
111    0
715    0
796    1
672    0
Name: Survived, dtype: int64

In [18]:
model.predict(X_test[0:10])

array([1, 0, 1, 1, 0, 0, 1, 0, 1, 0])

In [19]:
model.predict_proba(X_test[:10])

array([[3.89664825e-02, 9.61033517e-01],
       [8.14732202e-01, 1.85267798e-01],
       [1.01278016e-05, 9.99989872e-01],
       [3.45976058e-01, 6.54023942e-01],
       [9.73559891e-01, 2.64401089e-02],
       [9.74150257e-01, 2.58497428e-02],
       [3.36441469e-01, 6.63558531e-01],
       [9.70515624e-01, 2.94843757e-02],
       [7.30783343e-02, 9.26921666e-01],
       [9.22711762e-01, 7.72882382e-02]])

**Calculate the score using cross validation**

In [20]:
from sklearn.model_selection import cross_val_score
cross_val_score(GaussianNB(),X_train, y_train, cv=5)

array([0.792     , 0.808     , 0.8       , 0.80645161, 0.77419355])